In [2]:
from imutils.perspective import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import cv2 as cv
import imutils
import os

In [3]:
ARTIFACT_PATH = os.path.join(os.path.abspath(".."),"artifacts")

In [20]:
image = cv.imread(os.path.join(ARTIFACT_PATH,"results","License_Plate","plate0.jpg"))
ratio = image.shape[0]/150
orig = image.copy()
image = imutils.resize(image, height=150)

gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
gray = cv.GaussianBlur(gray, (5,5), 0)
edged = cv.Canny(gray, 150, 255)

print("STEP 1: Edge Detection")
cv.imshow("Image", image)
cv.imshow("Edged", edged)
cv.waitKey(0)
cv.destroyAllWindows()

STEP 1: Edge Detection


In [21]:
cnts = cv.findContours(edged.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv.contourArea, reverse = True)[:5]
# loop over the contours
for c in cnts:
	# approximate the contour
	peri = cv.arcLength(c, True)
	approx = cv.approxPolyDP(c, 0.02 * peri, True)
	# if our approximated contour has four points, then we
	# can assume that we have found our screen
	if len(approx) == 4:
		screenCnt = approx
		break
# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv.imshow("Outline", image)
cv.waitKey(0)
cv.destroyAllWindows()

STEP 2: Find contours of paper


In [29]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv.cvtColor(warped, cv.COLOR_BGR2GRAY)
# T = threshold_local(warped, 11, offset = 10, method = "gaussian")
# warped = (warped > T).astype("uint8") * 255
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv.imshow("Original", imutils.resize(orig, height = 50))
cv.imshow("Scanned", imutils.resize(warped, height = 50))
cv.waitKey(0)
cv.destroyAllWindows()

STEP 3: Apply perspective transform
